System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 11 × Apple M3 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, westmere)
Threads: 1 default, 0 interactive, 1 GC (on 11 virtual cores)


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/GitHub/biostat-m257-2024-spring/hw7`


Status `~/Documents/GitHub/biostat-m257-2024-spring/hw7/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [31c24e10] Distributions v0.25.109
  [295af30f] Revise v3.5.14
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


Again we continue with the linear mixed effects model (LMM)
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T.
$$
Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2).
$$

In HW6, we used the nonlinear programming (NLP) approach (Newton type algorithms) for optimization. In this assignment, we derive and implement an expectation-maximization (EM) algorithm for the same problem.

In [3]:
# load necessary packages; make sure install them first
using BenchmarkTools, Distributions, LinearAlgebra, Random, Revise

## Q1. (10 pts) Refresher on normal-normal model

Assume the conditional distribution
$$
\mathbf{y} \mid \boldsymbol{\gamma} \sim N(\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma}, \sigma^2 \mathbf{I}_n)
$$
and the prior distribution
$$
\boldsymbol{\gamma} \sim N(\mathbf{0}_q, \boldsymbol{\Sigma}).
$$
By the Bayes theorem, the posterior distribution is
\begin{eqnarray*}
f(\boldsymbol{\gamma} \mid \mathbf{y}) &=& \frac{f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})}{f(\mathbf{y})}, 
\end{eqnarray*}
where $f$ denotes corresponding density. 

Show that the posterior distribution of random effects $\boldsymbol{\gamma}$ is a multivariate normal with mean
\begin{eqnarray*}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&=& \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} (\mathbf{y} - \mathbf{X} \boldsymbol{\beta})
\end{eqnarray*}
and covariance
\begin{eqnarray*}
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1} \\
&=& \boldsymbol{\Sigma} - \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} \mathbf{Z} \boldsymbol{\Sigma}.
\end{eqnarray*}

### Solution:

As the variable $\gamma$ only appears in the numerator term, we can derive that
$$
\begin{eqnarray*}
f(\boldsymbol{\gamma} \mid \mathbf{y}) &=& \frac{f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})}{f(\mathbf{y})} \propto f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})
\end{eqnarray*}.
$$
Now as we have $\mathbf{y} \mid \boldsymbol{\gamma} \sim N(\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma}, \sigma^2 \mathbf{I}_n)$ and $\boldsymbol{\gamma} \sim N(\mathbf{0}_q, \boldsymbol{\Sigma})$, we can derive that
$$
\begin{eqnarray*}
f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma}) &=& (2\pi)^{-n/2} |\sigma^2 \mathbf{I}|^{-1/2} exp\bigg[-\frac{1}{2}(\mathbf{y} - \mathbf{X}\mathbb{\beta} - \mathbf{Z\gamma})^T (\sigma^2\mathbf{I})^{-1}(\mathbf{y} - \mathbf{X}\mathbb{\beta} - \mathbf{Z\gamma}) \bigg] 
\\
& &\times (2\pi)^{-q/2} |\mathbf{\Sigma}|^{-1/2} exp\bigg[-\frac{1}{2} \mathbf{\gamma}^T  \mathbf{\Sigma}^{-1} \mathbf{\gamma} \bigg] 
\\
&=& (2\pi)^{-(n+q)/2} (\sigma^2)^{-1/2} |\mathbf{\Sigma}|^{-1/2} exp\bigg[-\frac{1}{2}\gamma^T(\mathbf{\Sigma}^{-1}+\frac{1}{\sigma^2}\mathbf{Z^T}\mathbf{Z})\gamma + \frac{1}{\sigma^2} (\mathbf{Y} - \mathbf{X} \beta)^T \mathbf{Z}\gamma 
\\
& &- \frac{1}{2\sigma^2}(\mathbf{Y} - \mathbf{X}\beta)^T (\mathbf{Y} - \mathbf{X}\beta) \bigg]. 
\end{eqnarray*}
$$
By checking the 216 lecture notes and online materials, it is a multivariate normal distribution form, where the mean and variance terms are
$$
\begin{eqnarray*}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&=& \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}),
\end{eqnarray*}
$$ and 
$$
\begin{eqnarray*}
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1} \\
&=& \boldsymbol{\Sigma} - \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} \mathbf{Z} \boldsymbol{\Sigma}.
\end{eqnarray*}
$$

## Q2. (20 pts) Derive EM algorithm

1. Write down the complete log-likelihood
$$
\sum_{i=1}^m \log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)
$$

2. Derive the $Q$ function (E-step).
$$
Q(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2 \mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}).
$$

3. Derive the EM (or ECM) update of $\boldsymbol{\beta}$, $\boldsymbol{\Sigma}$, and $\sigma^2$ (M-step). 

### Solution

1. The complete log-likelihood is
$$
\begin{eqnarray*}
\sum_{i=1}^m \ell(\mathbf{y}_i, \mathbf{\gamma}_i|\beta, \mathbf{\Sigma}, \sigma^2) &=& \sum_i \bigg[log\ f(\mathbf{y}_i | \gamma_i, \beta, \sigma^2) + log\ f(\gamma_i|\mathbf{\Sigma}) \bigg]
\\
&=& \sum_i \bigg[-\frac{n_i}{2}log2\pi - \frac{1}{2} log|\sigma^2\mathbf{I}_{n_i}| - \frac{1}{2}(\mathbf{y}_i - \mathbf{X}_i\beta - \mathbf{Z}_i\gamma_i)^T\sigma^{-2}\mathbf{\Sigma}(\mathbf{y}_i - \mathbf{X}_i\beta - \mathbf{Z}_i\gamma_i) - \frac{q}{2}log2\pi - \frac{1}{2}log|\mathbf{\Sigma}| - \frac{1}{2}\gamma_i^T\mathbf{\Sigma}^{-1}\gamma_i \bigg].
\end{eqnarray*}
$$

2. The Q function is 
$$
\begin{eqnarray*}
Q &=& \mathbb E_{\gamma_i|y_i, \beta^{(t)}, \mathbf{\Sigma^{(t)}}, \sigma^{2(t)}}(\ell(\mathbf{y}_i, \gamma_i|\beta,\mathbf{\Sigma},\sigma^2))
\\
&=& c + \sum_i \mathbb E\bigg[-\frac{1}{2}(\mathbf{y}_i - \mathbf{X}_i\beta - \mathbf{Z}_i\gamma_i)^T\sigma^{-2}\mathbf{\Sigma}(\mathbf{y}_i - \mathbf{X}_i\beta - \mathbf{Z}_i\gamma_i)\bigg] - \sum_i \mathbb E\bigg(\frac{1}{2}\gamma_i^T \mathbf{\Sigma}^{-1}\gamma_i \bigg) - \frac{n_i}{2}log\ \sigma^2 - \frac{1}{2}log|\mathbf{\Sigma}|
\\
&=& c + \sum_i -\frac{1}{2}\sigma^{-2}\bigg[(\mathbf{y}_i - \mathbf{X}_i\beta)^T(\mathbf{y}_i - \mathbf{X}_i\beta) - 2(\mathbf{y}_i - \mathbf{X}_i\beta)^T\mathbf{Z}_i\mathbb E(\gamma_i|...) + \mathbb E(\gamma_i^T\mathbf{Z}_i^T \mathbf{Z}_i \gamma_i|...) \bigg] - \frac{1}{2}\sum_i\mathbb E(\gamma_i^T\mathbf{\Sigma^{-1}}\gamma_i|...) -\frac{n_i}{2}log\ \sigma^2 - \frac{1}{2}log|\mathbf{\Sigma}|.
\end{eqnarray*}
$$

where $c = \sum_i\bigg(-\frac{n_i}{2}log\ 2\pi - \frac{q}{2}log\ 2\pi\bigg)$, $\mathbb E(\gamma_i^T\mathbf{Z}_i^T\mathbf{Z}_i\gamma_i) = tr(\mathbb E(\gamma_i\gamma_i^T)\mathbf{Z}_i^T\mathbf{Z}_i) = tr\bigg[(Var(\gamma_i) + \mathbb E(\gamma_i)\mathbb E(\gamma_i)^T)\mathbf{Z}_i^T\mathbf{Z}_i\bigg]$.

3. The update functions for model parameters are
$$
\frac{\partial Q}{\partial \beta} = \sum_i \sigma^2\bigg[\mathbf{X}_i^T(\mathbf{y}_i-\mathbf{X}_i\beta) - \mathbf{X}^T_i \mathbf{Z}_i \mathbb E(\gamma_i|...) \bigg].
$$
Let it to be 0, we get
$$
\hat\beta^{(t+1)} = \bigg(\sum_i\mathbf{X}_i^T \mathbf{X}_i \bigg)^{-1} \sum_i \mathbf{X}_i^T \bigg[\mathbf{y}_i - \mathbf{Z}_i \mathbb E(\gamma_i|...) \bigg].
$$

$$
\frac{\partial Q}{\partial \sigma^2} = -\frac{1}{2}\sum_i \bigg[\frac{n_i}{\sigma^2} - \frac{\mathbb E(\gamma_i^T \gamma_i)}{\sigma^4}\bigg].
$$
Let it to be 0, we get
$$
\hat\sigma^{2(t+1)} = \frac{\sum_i \mathbb E(\gamma_i^T\gamma_i)}{n}.
$$

$$
\frac{\partial Q}{\partial \mathbf{\Sigma}} = -\frac{m}{2}\mathbf{\Sigma}^{-1} + \frac{1}{2}\sum_i\mathbf{\Sigma}^{-1}\mathbb E(\gamma_i^T\gamma_i)\mathbf{\Sigma}^{-1}.
$$
Let it to be 0, we get
$$
\hat{\mathbf{\Sigma}}^{t+1} = \frac{1}{m} \sum_i \bigg[Var(\gamma_i) + \mathbb E(\gamma_i)\mathbb E(\gamma_i)^T \bigg].
$$

## Q3. (20 pts) Objective of a single datum

We modify the code from HW6 to evaluate the objective, the conditional mean of $\boldsymbol{\gamma}$, and the conditional variance of $\boldsymbol{\gamma}$. Start-up code is provided below. You do _not_ have to use this code.

In [4]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # posterior mean and variance of random effects γ
    μγ         :: Vector{T} # posterior mean of random effects
    νγ         :: Matrix{T} # posterior variance of random effects
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    rtr        :: Vector{T}
    xty        :: Vector{T}
    zty        :: Vector{T}
    ztr        :: Vector{T}
    ltztr      :: Vector{T}
    xtr        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ltztzl     :: Matrix{T}
    storage_qq :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y::Vector{T}, 
    X::Matrix{T}, 
    Z::Matrix{T}) where T <: AbstractFloat
    n, p, q = size(X, 1), size(X, 2), size(Z, 2)
    μγ         = Vector{T}(undef, q)
    νγ         = Matrix{T}(undef, q, q)
    yty        = abs2(norm(y))
    rtr        = Vector{T}(undef, 1)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    ztr        = similar(zty)
    ltztr      = similar(zty)
    xtr        = Vector{T}(undef, p)
    storage_p  = similar(xtr)
    storage_q  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ltztzl     = similar(ztz)
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, μγ, νγ, 
        yty, rtr, xty, zty, ztr, ltztr, xtr,
        storage_p, storage_q, 
        xtx, ztx, ztz, ltztzl, storage_qq)
end

LmmObs

In [5]:
"""
    logl!(obs::LmmObs, β, Σ, L, σ², updater = false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `Σ`, 
and `σ²`. The lower triangular Cholesky factor `L` of `Σ` must be supplied too.
The fields `obs.μγ` and `obs.νγ` are overwritten by the posterior mean and 
posterior variance of random effects. If `updater==true`, fields `obs.ztr`, 
`obs.xtr`, and `obs.rtr` are updated according to input parameter values. 
Otherwise, it assumes these three fields are pre-computed. 
"""
function logl!(
        obs     :: LmmObs{T}, 
        β       :: Vector{T}, 
        Σ       :: Matrix{T},
        L       :: Matrix{T},
        σ²      :: T,
        updater :: Bool = false
        ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################
    # form the q-by-q matrix: Lt Zt Z L
    copy!(obs.ltztzl, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.ltztzl) # O(q^3)        
    # form the q-by-q matrix: M = σ² I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ltztzl)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # Zt * res
    updater && BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.ztr, obs.zty)) # O(pq)
    # Lt * (Zt * res)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.ltztr, obs.ztr))    # O(q^2)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, copy!(obs.storage_q, obs.ltztr)) # O(q^3)
    # Xt * res = Xt * y - Xt * X * β
    updater && BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.xtr, obs.xty))
    # l2 norm of residual vector
    updater && (obs.rtr[1] = obs.yty - dot(obs.xty, β) - dot(obs.xtr, β))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q # log det term
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (obs.rtr[1] - qf) * σ²inv 
    logl /= -2
    ######################################
    # TODO: Evaluate posterior mean and variance
    ######################################    
    
    #νγ
    obs.νγ .= 0
    for i in 1:q
       obs.νγ[i, i] = 1
    end
    BLAS.trsm!('L', 'L', 'N', 'N', 1.0, L, obs.νγ)
    BLAS.trsm!('L', 'L', 'T', 'N', 1.0, L, obs.νγ)
    BLAS.axpy!(σ²inv, obs.ztz, obs.νγ)
    LAPACK.potrf!('L', obs.νγ)
    LAPACK.potri!('L', obs.νγ)
    LinearAlgebra.copytri!(obs.νγ, 'L')
    
    #μγ
    BLAS.gemm!('N', 'N', T(σ²inv), obs.νγ, obs.ztr, T(0), obs.μγ)
    
    ###################
    # Return
    ###################        
    return logl
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/posterior mean/var evaluator here. It's the same test datum as in HW3 and HW6.

In [6]:
Random.seed!(257)

# dimension
n, p, q = 2000, 5, 3
# predictors
X = [ones(n) randn(n, p - 1)]
Z = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [7]:
@show logl = logl!(obs, β, Σ, L, σ², true)
@show obs.μγ
@show obs.νγ;

logl = logl!(obs, β, Σ, L, σ², true) = -3256.17933580583
obs.μγ = [0.10608689301332866, -0.2510419060257252, -1.4653979409850377]
obs.νγ = [0.0007494356395786613 -1.2183420464422407e-6 -2.176783682941756e-6; -1.2183420464422407e-6 0.0007542331466357765 2.155346461246878e-5; -2.176783682941756e-6 2.155346461246878e-5 0.0007465271344917232]


You will lose all 20 points if following statement throws `AssertionError`.

In [8]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.μγ - [0.10608689301333621, 
        -0.25104190602577225, -1.4653979409855415]) < 1e-4
@assert norm(obs.νγ - [
        0.0007494356395909563 -1.2183420093769967e-6 -2.176783643112221e-6; 
        -1.2183420282298223e-6 0.0007542331467601107 2.1553464632686345e-5; 
        -2.1767836636008638e-6 2.1553464641863096e-5 0.0007465271342535443
        ]) < 1e-4

### Efficiency

Benchmark for efficiency.

In [9]:
bm_obj = @benchmark logl!($obs, $β, $Σ, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 71 evaluations.
 Range (min … max):  822.183 ns …  1.333 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     900.831 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   899.060 ns ± 39.958 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▃▅▅▅▃▃▂▃▂▁▁▁ ▁▃▄▇██▇▆▅▄▃▂▂▂▁▁                              ▂
  ▃█████████████████████████████████▇▇▇▇█▇▆▆▅▅▅▇▆▇▇▇▇▆▇▆▇▆▆▄▆▅ █
  822 ns        Histogram: log(frequency) by time      1.06 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median runt time is 800ns. You will get full credit if the median run time is within 10μs. The points you will get are

In [10]:
clamp(10 / (median(bm_obj).time / 1e3) * 10, 0, 10)

10.0

In [11]:
# # check for type stability
# @code_warntype logl!(obs, β, Σ, L, σ²)

In [12]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, Σ, L, σ²); end
# Profile.print(format=:flat)

## Q4. LmmModel type

We modify the `LmmModel` type in HW6 to hold all data points, model parameters, and intermediate arrays.

In [13]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat}
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β      :: Vector{T}
    Σ      :: Matrix{T}
    L      :: Matrix{T}
    σ²     :: Vector{T}    
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty    :: Vector{T}
    xtr    :: Vector{T}
    ztr2   :: Vector{T}
    xtxinv :: Matrix{T}
    ztz2   :: Matrix{T}
    term1  :: Vector{T}
    term2  :: Vector{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p      = size(obsvec[1].X, 2)
    q      = size(obsvec[1].Z, 2)
    # parameters
    β      = Vector{T}(undef, p)
    Σ      = Matrix{T}(undef, q, q)
    L      = Matrix{T}(undef, q, q)
    σ²     = Vector{T}(undef, 1)    
    # intermediate arrays
    xty    = zeros(T, p)
    xtr    = similar(xty)
    ztr2   = Vector{T}(undef, abs2(q))
    xtxinv = zeros(T, p, p)
    term1  = Vector{T}(undef, p)
    term2  = Vector{T}(undef, q)
    # pre-calculate \sum_i Xi^T Xi and \sum_i Xi^T y_i
    @inbounds for i in eachindex(obsvec)
        obs = obsvec[i]
        BLAS.axpy!(T(1), obs.xtx, xtxinv)
        BLAS.axpy!(T(1), obs.xty, xty)
    end
    # invert X'X
    LAPACK.potrf!('U', xtxinv)
    LAPACK.potri!('U', xtxinv)
    LinearAlgebra.copytri!(xtxinv, 'U')
    ztz2   = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, Σ, L, σ², xty, xtr, ztr2, xtxinv, ztz2, term1, term2)
end

LmmModel

## Q5. Implement EM update

Let's write the key function `update_em!` that performs one iteration of EM update.

In [14]:
"""
    update_em!(m::LmmModel, updater::Bool = false)

Perform one iteration of EM update. It returns the log-likelihood calculated 
from input `m.β`, `m.Σ`, `m.L`, and `m.σ²`. These fields are then overwritten 
by the next EM iterate. The fields `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` are updated according to the resultant `m.β`. If `updater==true`, 
the function first updates `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` according to `m.β`. If `updater==false`, it assumes these fields 
are pre-computed.
"""
function update_em!(m::LmmModel{T}, updater::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    m.Σ .= 0 .* m.Σ
    m.term1 .= 0 .* m.term1
    M = length(m.data)
    n_total = 0
    
    #update m.β
    for i in 1:M
        logl += logl!(m.data[i], m.β, m.Σ, m.L, m.σ²[1], updater)
        BLAS.gemv!('T', T(-1), m.data[i].ztx, m.data[i].μγ, T(1), m.term1)
    end
    BLAS.axpy!(T(1), m.xty, m.term1)
    BLAS.gemv!('N', T(1), m.xtxinv, m.term1, T(0), m.β)
    
    
    for i in 1:M
        #update m.data[i].ztr, m.data[i].xtr, m.data[i].rtr
        if updater           
            copy!(m.data[i].ztr, m.data[i].zty)
            copy!(m.data[i].xtr, m.data[i].xty)
            BLAS.gemv!('N', T(-1), m.data[i].ztx, m.β, T(1), m.data[i].ztr)
            BLAS.gemv!('N', T(-1), m.data[i].xtx, m.β, T(1), m.data[i].xtr)
            m.data[i].rtr[1] = m.data[i].yty - dot(m.data[i].xty, m.β) - dot(m.data[i].xtr, m.β)
        end
        # update m.σ²  
        m.σ²[1] += m.data[i].rtr[1]
        m.σ²[1] += dot(m.data[i].ztz, m.data[i].νγ)
        BLAS.gemv!('N', T(1), m.data[i].ztz, m.data[i].μγ, T(0), m.term2)
        BLAS.axpy!(T(-2), m.data[i].ztr, m.term2)
        m.σ²[1] += dot(m.term2, m.data[i].μγ)
        n_total += size(m.data[i].X, 1)
        # update m.Σ    
        BLAS.axpy!(T(1), m.data[i].νγ, m.Σ)
        BLAS.gemm!('N', 'T', T(1), m.data[i].μγ, m.data[i].μγ, T(1), m.Σ)
    end
    m.σ²[1] /= n_total
    m.Σ ./= M
    
    
    # update m.L
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    
    # return log-likelihood at input parameter values
    logl
end

update_em!

## Q6. (30 pts) Test data

Let's generate a synthetic longitudinal data set (same as HW6) to test our algorithm.

In [15]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

### Correctness

Evaluate log-likelihood and gradient at the true parameter values.

In [16]:
copy!(lmm.β, βtrue)
copy!(lmm.Σ, Σtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj1 = update_em!(lmm, true)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²
println()
@show obj2 = update_em!(lmm, false)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

obj1 = update_em!(lmm, true) = -2.8400684383699712e6
lmm.β = [0.10003613673625014, 6.500382871080185, -3.4998646342112245, 0.9997124657606653, 4.999230851463542]
lmm.Σ = [1.990388276045525 0.06862095707037442 0.053472901794726566; 0.06862095707037442 1.2813220461216892 -0.09044913324906331; 0.053472901794726566 -0.09044913324906331 0.9435400745724168]
lmm.L = [1.410811212049835 0.06862095707037442 0.053472901794726566; 0.04863936186803602 1.1309094829378519 -0.09044913324906331; 0.03790223761904559 -0.08160924927472397 0.9671832429217858]
lmm.σ² = [1.4987375875345585]

obj2 = update_em!(lmm, false) = -2.840060460542926e6
lmm.β = [0.10007136573194023, 6.500383550631406, -3.499864298040665, 0.9997119269501409, 4.999229480978843]
lmm.Σ = [1.9903775375793393 0.06870107684162093 0.05354351750429605; 0.06870107684162093 1.2814409208767201 -0.09059223996845789; 0.05354351750429605 -0.09059223996845789 0.9434431683176685]
lmm.L = [1.4108074062675384 0.06870107684162093 0.05354351750429605; 0.0

1-element Vector{Float64}:
 1.4987354160243331

Test correctness. You will loss all 30 points if following code throws `AssertError`.

In [17]:
@assert abs(obj1 - (-2.840068438369969e6)) < 1e-4
@assert abs(obj2 - (-2.84006046054206e6)) < 1e-4

### Efficiency

Test efficiency of EM update.

In [18]:
bm_emupdate = @benchmark update_em!($lmm, true) setup=(
    copy!(lmm.β, βtrue);
    copy!(lmm.Σ, Σtrue);
    copy!(lmm.L, Ltrue);
    lmm.σ²[1] = σ²true)

BenchmarkTools.Trial: 3935 samples with 1 evaluation.
 Range (min … max):  1.239 ms …  1.366 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.253 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.269 ms ± 30.117 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▆▇█▆▄▁                                                   
  ▂▄▇██████▇▆▅▄▄▃▃▃▃▂▂▁▁▁▁▁▁▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▃▄▆▇▆▅▄▄▃▃▂▂▂ ▃
  1.24 ms        Histogram: frequency by time        1.33 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 1ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [19]:
clamp(10 / (median(bm_emupdate).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [20]:
clamp(10 - median(bm_emupdate).memory / 100, 0, 10)

10.0

## Q7. Starting point

We use the same least squares estimates as in HW6 as starting point. 

In [21]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # LS estimate for β
    mul!(m.β, m.xtxinv, m.xty)
    # LS etimate for σ2 and Σ
    rss, ntotal = zero(T), 0
    fill!(m.ztz2, 0)
    fill!(m.ztr2, 0)    
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        ntotal += length(obs.y)
        # update Xt * res
        BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # rss of i-th individual
        rss += obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
        # update Zi' * res
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty))
        # Zi'Zi ⊗ Zi'Zi
        kron_axpy!(obs.ztz, obs.ztz, m.ztz2)
        # Zi'res ⊗ Zi'res
        kron_axpy!(obs.ztr, obs.ztr, m.ztr2)
    end
    m.σ²[1] = rss / ntotal
    # LS estimate for Σ = LLt
    LAPACK.potrf!('U', m.ztz2)
    BLAS.trsv!('U', 'T', 'N', m.ztz2, m.ztr2)
    BLAS.trsv!('U', 'N', 'N', m.ztz2, m.ztr2)
    copyto!(m.Σ, m.ztr2)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    for j in 2:q, i in 1:j-1
        m.L[i, j] = 0
    end
    m
end

"""
    kron_axpy!(A, X, Y)

Overwrite `Y` with `A ⊗ X + Y`. Same as `Y += kron(A, X)` but
more memory efficient.
"""
function kron_axpy!(
        A::AbstractVecOrMat{T},
        X::AbstractVecOrMat{T},
        Y::AbstractVecOrMat{T}
        ) where T <: Real
    m, n = size(A, 1), size(A, 2)
    p, q = size(X, 1), size(X, 2)
    @assert size(Y, 1) == m * p
    @assert size(Y, 2) == n * q
    @inbounds for j in 1:n
        coffset = (j - 1) * q
        for i in 1:m
            a = A[i, j]
            roffset = (i - 1) * p            
            for l in 1:q
                r = roffset + 1
                c = coffset + l
                for k in 1:p                
                    Y[r, c] += a * X[k, l]
                    r += 1
                end
            end
        end
    end
    Y
end

kron_axpy!

In [22]:
init_ls!(lmm)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

lmm.β = [0.18207934611476317, 6.500480700993724, -3.4979107842091604, 1.0011132962297955, 5.000251985791927]
lmm.Σ = [1.9794302836685052 0.07258461003916686 0.05717147035274017; 0.07258461003916687 1.284038573476771 -0.07707942768978554; 0.05717147035274017 -0.07707942768978555 0.9509885905046903]
lmm.L = [1.4069222734993234 0.0 0.0; 0.05159105901325528 1.131979211870369 0.0; 0.04063584138912111 -0.06994463586493138 0.9718256360134829]
lmm.σ² = [5.709004733413665]


1-element Vector{Float64}:
 5.709004733413665

## Q8. Estimation by EM

We write a function `fit!` that implements the EM algorithm for estimating LMM.

In [23]:
"""
    fit!(m::LmmModel)

Fit an `LmmModel` object by MLE using a EM algorithm. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m       :: LmmModel;
        maxiter :: Integer       = 10_000,
        ftolrel :: AbstractFloat = 1e-12,
        prtfreq :: Integer       = 0
    )
    obj = update_em!(m, true)
    for iter in 0:maxiter
        obj_old = obj
        # EM update
        obj = update_em!(m, false)
        # print obj
        prtfreq > 0 && rem(iter, prtfreq) == 0 && println("iter=$iter, obj=$obj")
        # check monotonicity
        obj < obj_old && (@warn "monotoniciy violated")
        # check convergence criterion
        (obj - obj_old) < ftolrel * (abs(obj_old) + 1) && break
        # warning about non-convergence
        iter == maxiter && (@warn "maximum iterations reached")
    end
    m
end


fit!

## Q9. (20 pts) Test drive

Now we can run our EM algorithm to compute the MLE.

In [24]:
# initialize from least squares
init_ls!(lmm)

@time fit!(lmm, prtfreq = 1);

println("objective value at solution: ", update_em!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)

iter=0, obj=-2.840068891274902e6
iter=1, obj=-2.840058786808006e6
iter=2, obj=-2.8400587867737473e6
iter=3, obj=-2.8400587867737347e6
  0.020934 seconds (27.10 k allocations: 1.951 MiB, 74.00% compilation time)
objective value at solution: -2.8400587867737487e6

solution values:
lmm.β = [0.18207952873848723, 6.500383547102793, -3.499864282266057, 0.9997119324209937, 4.999229484614066]
lmm.σ² = [1.498735412396533]
lmm.L * transpose(lmm.L) = [1.9910120962551912 0.13511167124776668 0.10338312791979659; 0.13511167124776668 1.2896479941488803 -0.17820185549896944; 0.10338312791979659 -0.17820185549896944 0.9434430216800316]


3×3 Matrix{Float64}:
 1.99101    0.135112   0.103383
 0.135112   1.28965   -0.178202
 0.103383  -0.178202   0.943443

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [25]:
# objective at solution should be close enough to the optimal
@assert update_em!(lmm) > -2.840059e6

### Efficiency

My median run time 5ms. You get 10 points if your median run time is within 1s.

In [26]:
bm_em = @benchmark fit!($lmm) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 979 samples with 1 evaluation.
 Range (min … max):  4.805 ms …  5.073 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.934 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.935 ms ± 19.645 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                    ▁▁▃▄██▅▇▅▆                
  ▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▂▂▃▃▃▄▄▅▇███████████▇▇▅▅▃▃▂▂▂▂▂▂▂ ▃
  4.81 ms        Histogram: frequency by time        4.99 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [27]:
# this is the points you get
clamp(1 / (median(bm_em).time / 1e9) * 10, 0, 10)

10.0

## Q10. (10 pts) EM vs Newton type algorithms

Contrast EM algorithm to the Newton type algorithms (gradient free, gradient based, using Hessian) in HW6, in terms of the stability, convergence rate (how fast the algorithm is converging),  final objective value, total run time, derivation, and implementation efforts. 

### Solution:

1. Stability: from the lecture notes, we should notice that EM algorithm relies on a monotonically increasing objective function, while Newton type algorithms require designed choices on the Hessian matrix. The former one hence provides more stability on parameter convergence.
2. Convergence rate and total run time: in fact, the total run time of EM algorithm sometimes highly depends on the choice of starting point, while the convergence rate of Newton type algorithms depends on the choice of approximation of the Hessian matrix.
3. Final objective value: in above example, the final objective value of EM algorithm is -2.8400587867737487e6, which of Newton type algorithm returned by HW 6 is -2.8400684383699694e6.
4. Derivation: in my view, EM algorithm requires an appropriate design for the unknown parameters and the Q function, which in fact highly relies on the designer's knowledge and experience. However, Newton type algorithm is also not that easy for the users, as the choice of the approximation of Hessian matrix also determines the convergence rate and total run time.
5. Implementation: for the example given in homework assignments, the implementation are both easy to achieve. However, in other real cases, some higher-level optimization techniques may be used in Newton type algorithms.